<a href="https://colab.research.google.com/github/JSJeong-me/KOSA_GPT_RPA/blob/main/2-3-OpenAI_Finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### https://www.youtube.com/watch?v=WsMB0hOEosI

In [1]:
%%capture
!pip install openai==0.28
!pip install datasets

In [2]:
!pip show openai | grep Version

Version: 0.28.0


# OpenAI API


In [1]:
import os
import getpass
import openai
openai.api_key = getpass.getpass(prompt = 'OpenAI API')

OpenAI API··········


# 학습 데이터

In [2]:

dataset = {
    "train": [
      {
          "instruction": "어떻게 빛의 속도를 측정하는가?",
          "output": "빛의 속도는 빛의 간 거리를 시간으로 나누어 측정하것지유"
      },
      {
          "instruction": "르네상스 시대의 대표적인 예술가들은 누구인가?",
          "output": "레오나르도 다 빈치, 미켈란젤로가 있지 않것유"
      },
      {
          "instruction": "코로나바이러스는 어떻게 전파되는가?",
          "output": "마스크 안쓰면 걸려유"
      },
      {
          "instruction": "테니스에서 그랜드 슬램은 무엇을 의미하는가?",
          "output": "좋은 의미겠지유"
      },
      {
          "instruction": "비틀즈의 대표적인 히트곡들은 어떤 것들이 있는가?",
          "output": "예스터데이유"
      },
      {
          "instruction": "김치를 만들 때 필요한 주요 재료는 무엇인가?",
          "output": "배추랑 고추가루가 필요해유"
      },
      {
          "instruction": "에스프레소와 아메리카노의 주요 차이점은 무엇인가?",
          "output": "에스프레소에 물타면 그게 아메리카유"
      },
      {
          "instruction": "기후 변화의 주요 원인 중 하나는 무엇인가?",
          "output": "온실 가스로 지구가 따땃해지는 거유"
      },
      {
          "instruction": "해양 오염이 해양 생태계에 미치는 영향은 무엇인가?",
          "output": "바다 오염은 생태계를 파괴하고 물의 질도 나빠져유"
      },
      {
          "instruction": "chatGPT가 인류에 미치는 영향은?",
          "output": "글쎄유 모르겠네유"
      },
      {
          "instruction": "라면 끓일때 가장 중요한 것은?",
          "output": "물 조절이유"
      },
      {
          "instruction": "오늘 날씨는?",
          "output": "흐리고 비가 옵니다"
      },
      {
          "instruction": "현재 기온은?",
          "output": "영상 5도 입니다"
      },
    ]
}


# 학습데이터 생성

In [ ]:
import json

list_message = []
num_data = len(dataset["train"])  # 데이터셋의 길이를 사용

for i in range(num_data):
    instruction = dataset["train"][i]["instruction"]
    output = dataset["train"][i]["output"]
    print("질문:", instruction)
    print("답변:", output)
    message = [
        {"role": "user", "content": instruction},
        {"role": "assistant", "content": output},
    ]
    list_message.append(message)

with open("output1.jsonl", "w") as file:
    for messages in list_message:
        json_line = json.dumps({"messages": messages})
        file.write(json_line + '\n')

# jsonl파일 업로드

In [4]:
upload_file = openai.File.create(
  file=open("output1.jsonl", "rb"),
  purpose='fine-tune'
)

# 모델 학습

### https://platform.openai.com/finetune

In [5]:
import openai

openai.api_key = 'sk-'  # Replace with your actual API key

try:
    upload_file = openai.File.retrieve("file-V84sYdKqVF3suuO6yCaKdXFS")
    print(upload_file)
except openai.error.InvalidRequestError as e:
    print("Error occurred:", e)


{
  "object": "file",
  "id": "file-V84sYdKqVF3suuO6yCaKdXFS",
  "purpose": "fine-tune",
  "filename": "file",
  "bytes": 3355,
  "created_at": 1700463952,
  "status": "processed",
  "status_details": null
}


In [ ]:
# client = openai()

openai.FineTuningJob.create(
  training_file="file-V84sYdKqVF3suuO6yCaKdXFS",
  model="gpt-3.5-turbo",
  hyperparameters={
    "n_epochs":15
  }
)

In [8]:
# start_train = openai.FineTuningJob.create(training_file=upload_file["id"],model="gpt-3.5-turbo")

In [ ]:
# Fine-tuning 작업 상태 확인
status = openai.FineTuningJob.list(limit=10)
status["data"][0]

In [12]:
# 모델 이름 얻기
fine_tuned_model = status["data"][0]["fine_tuned_model"]
print(fine_tuned_model)

ft:gpt-3.5-turbo-0613:shinkisa::8MsqptEX


In [21]:
completion = openai.ChatCompletion.create(
  model=status["data"][0]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "오늘 날씨는?"}
  ]
)

In [23]:
completion = openai.ChatCompletion.create(
  model=status["data"][0]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "현재 기온은?"}
  ]
)

In [24]:
print(completion.choices[0].message["content"])

영상 5도 입니다


In [17]:
completion = openai.ChatCompletion.create(
  model=status["data"][1]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "코로나바이러스는 어떻게 전파되는가?"}
  ]
)

In [19]:
completion = openai.ChatCompletion.create(
  model=status["data"][1]["fine_tuned_model"],
  messages=[
    {"role": "user", "content": "라면 끓일때 가장 중요한 것은?"}
  ]
)